In [ ]:
testBitstring = '01001110100010010011'

In [ ]:
import itertools
import random
import math

In [ ]:
def bits_for_series(n):
    """
    Количество бит по Хартли:
    floor(log2(n!))
    """
    if n < 2:
        return 0
    return int(math.log2(math.factorial(n)))


In [ ]:
def split_into_blocks(data):
    """
    Возвращает список блоков:
    ('series', [элементы]) или ('term', элемент)
    """
    blocks = []
    current = []

    for x in data:
        if x not in current:
            current.append(x)
        else:
            if len(current) > 1:
                blocks.append(('series', current))
            else:
                blocks.append(('term', current[0]))
            blocks.append(('term', x))
            current = []

    if current:
        if len(current) > 1:
            blocks.append(('series', current))
        else:
            blocks.append(('term', current[0]))

    return blocks


In [ ]:
def get_allowed_permutations(series, salt):
    """
    Отбирает 2^k допустимых перестановок из n!
    """
    n = len(series)
    k = bits_for_series(n)
    count = 2 ** k

    all_perms = list(itertools.permutations(series))
    rnd = random.Random(salt + n)
    rnd.shuffle(all_perms)

    return all_perms[:count]


In [ ]:
def embed_series(series, secret_bits, salt):
    n = len(series)
    k = bits_for_series(n)

    if k == 0 or len(secret_bits) < k:
        return series

    # Серия длины 2 → 1 бит
    if n == 2:
        bit = secret_bits.pop(0)
        return series if bit == '0' else series[::-1]

    perms = get_allowed_permutations(series, salt)

    bits = ''.join(secret_bits[:k])
    del secret_bits[:k]

    idx = int(bits, 2)
    return list(perms[idx])


In [ ]:
def extract_series_bits(stego_series, original_series, salt):
    n = len(original_series)
    k = bits_for_series(n)

    if k == 0:
        return ""

    if n == 2:
        return '0' if stego_series == original_series else '1'

    perms = get_allowed_permutations(original_series, salt)
    idx = perms.index(tuple(stego_series))

    return format(idx, f'0{k}b')


In [ ]:
# Version 2
def extract_series_bits_safe(stego_series, original_series, salt):
    n = len(original_series)
    k = bits_for_series(n)

    if k == 0:
        return ""

    if n == 2:
        return '0' if stego_series == original_series else '1'

    perms = get_allowed_permutations(original_series, salt)

    stego_tuple = tuple(stego_series)
    if stego_tuple not in perms:
        # Серия не использовалась для встраивания
        return ""

    idx = perms.index(stego_tuple)
    return format(idx, f'0{k}b')

In [ ]:
def SMT_embed(M, S, salt):
    secret_bits = list(S)
    blocks = split_into_blocks(M)

    output = []

    for kind, block in blocks:
        if kind == 'term':
            output.append(block)
            continue

        need = bits_for_series(len(block))

        if len(secret_bits) < need:
            output.extend(block)
            continue

        embedded = embed_series(block, secret_bits, salt)
        output.extend(embedded)

    return {
        'output': output,
        'blocks': blocks
    }


In [ ]:
def SMT_extract(M_original, M_stego, salt):
    blocks_orig = split_into_blocks(M_original)
    blocks_stego = split_into_blocks(M_stego)

    secret_bits = []

    for (k1, b1), (k2, b2) in zip(blocks_orig, blocks_stego):
        if k1 == 'series' and k2 == 'series' and len(b1) == len(b2):
            secret_bits.append(
                extract_series_bits(b2, b1, salt)
            )

    return {
        'extracted': ''.join(secret_bits),
        'blocks_orig': blocks_orig,
        'blocks_stego': blocks_stego
    }


In [ ]:
def SMT_capacity(M):
    blocks = split_into_blocks(M)
    cap = 0
    for kind, block in blocks:
        if kind == 'series':
            cap += bits_for_series(len(block))
    return cap

In [ ]:
# M = [
#     "00","01","10","11",
#     "10","00","01","10",
#     "00","10","01","11",
#     "00","01","11","00",
#     "10","00","01","11",
#     "00","01","10","00",
#     "00","01","10","11"
# ]

In [ ]:
M = [
    "00","01","10","11",
    "10","00","01","11",
    "00","10","01","11"
]
S = "1011001110001011"
salt = 12345
stego = SMT_embed(M, S, salt)
extracted = SMT_extract(M, stego['output'], salt)
print("Capacity:", SMT_capacity(M), "bits")
print("Original: ", M)
print("Stego:    ", stego['output'])
print("Secret:   ", S)
print("Extracted:", extracted['extracted'][:len(S)])

In [ ]:
print(f'Stego blocks:\n {stego['blocks']}')

print(f'Extracted blocks_orig:\n {extracted['blocks_orig']}')
print(f'Extracted blocks_stego:\n {extracted['blocks_stego']}')

## Анализ

### Частотность

In [ ]:
from collections import Counter

def element_frequencies(container):
    """
    Возвращает частоты элементов контейнера.
    """
    return Counter(container)

freq_orig = element_frequencies(M)
freq_stego = element_frequencies(stego['output'])

print("Частоты исходного:", freq_orig)
print("Частоты стего:", freq_stego)

### Биграммы

In [ ]:
def bigram_frequencies(container):
    """
    Возвращает частоты биграмм контейнера.
    """
    bigrams = [
        (container[i], container[i + 1])
        for i in range(len(container) - 1)
    ]
    return Counter(bigrams)

bi_orig = bigram_frequencies(M)
bi_stego = bigram_frequencies(stego['output'])

print("Биграммы исходного:", bi_orig)
print("Биграммы стего:", bi_stego)

### Энтропия

In [ ]:
import math

def entropy_from_frequencies(freqs):
    """
    Вычисляет энтропию Шеннона по частотам.
    """
    total = sum(freqs.values())
    entropy = 0.0

    for count in freqs.values():
        p = count / total
        entropy -= p * math.log2(p)

    return entropy

H_orig = entropy_from_frequencies(freq_orig)
H_stego = entropy_from_frequencies(freq_stego)

print("Энтропия исходного:", H_orig)
print("Энтропия стего:", H_stego)

In [ ]:
import matplotlib.pyplot as plt

def plot_element_frequencies(original, stego):
    freq_orig = element_frequencies(original)
    freq_stego = element_frequencies(stego)

    elements = sorted(freq_orig.keys())
    orig_vals = [freq_orig[e] for e in elements]
    stego_vals = [freq_stego[e] for e in elements]

    x = range(len(elements))

    plt.figure()
    plt.bar(x, orig_vals)
    plt.bar(x, stego_vals, bottom=orig_vals)
    plt.xticks(x, elements)
    plt.title("Частоты элементов контейнера")
    plt.xlabel("Элемент")
    plt.ylabel("Частота")
    plt.show()

plot_element_frequencies(M, stego['output'])

In [ ]:
def plot_entropy(original, stego):
    H_orig = entropy_from_frequencies(element_frequencies(original))
    H_stego = entropy_from_frequencies(element_frequencies(stego))

    plt.figure()
    plt.bar(["Исходный", "Стего"], [H_orig, H_stego])
    plt.ylabel("Энтропия (бит)")
    plt.title("Энтропия контейнера")
    plt.show()

plot_entropy(M, stego['output'])

In [ ]:
def bigram_percentages(container):
    bigrams = bigram_frequencies(container)
    total = sum(bigrams.values())

    return {
        k: (v / total) * 100
        for k, v in bigrams.items()
    }

def compare_bigram_percentages(original, stego):
    bi_orig = bigram_percentages(original)
    bi_stego = bigram_percentages(stego)

    all_keys = set(bi_orig) | set(bi_stego)

    diff = {}
    for k in all_keys:
        diff[k] = bi_stego.get(k, 0) - bi_orig.get(k, 0)

    return diff

diff = compare_bigram_percentages(M, stego['output'])

for k, v in list(diff.items())[:10]:
    print(f"{k}: {v:.3f}%")